# InfluxDB 
## Purpose: Test InfluxDB in Jupyter
## Author: mjz

In [2]:
# https://github.com/influxdata/influxdb-client-python/tree/master/notebooks

import os
import sys

sys.path.insert(0, os.path.abspath('../'))



In [10]:
from __future__ import print_function

import math
import os
import pandas as pd
import time
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from influxdb import InfluxDBClient
from influxdb import DataFrameClient


In [27]:
config = {
        'host':     '192.168.0.115',
        'port':       8086,
        'database': 'perfkit',
        'username':     'perfkit',
        'password': 'perfkit',

    }

In [3]:
%%bash
python --version
# /opt/projects/diss/py-mulval/src/run_boromir.py --influx_uri='http://influx1:8086'

Python 3.6.8


In [5]:
client = InfluxDBClient('192.168.0.115', '8086')
# client.create_database(pkb)
dbs = client.get_list_database()
print(dbs)
# client.create_database('example')
# client.switch_database('example')


client.query('show databases')


[{'name': 'sample_database'}, {'name': '_internal'}, {'name': 'perfkit'}]


ResultSet({'('databases', None)': [{'name': 'sample_database'}, {'name': '_internal'}, {'name': 'perfkit'}]})

In [7]:
sql = r"""
SELECT mean("value") AS "mean_value" 
FROM "perfkit"."autogen"."perfkitbenchmarker" 
WHERE time > :dashboardTime: 
AND time < :upperDashboardTime: 
GROUP BY time(:interval:) FILL(null)
"""

In [32]:
sql = r"""
SELECT * 
FROM "perfkit"."autogen"."perfkitbenchmarker"
"""
client = DataFrameClient(**config)

print("Create pandas DataFrame")
# df = pd.DataFrame(data=list(range(30)),
#                   index=pd.date_range(start='2014-11-16',
#                                       periods=30, freq='H'), columns=['0'])

client = DataFrameClient(host     = '192.168.0.115',
                         port     = 8086,
                         database = 'perfkit')

# result = client.query('select * from ' + measurement + ' where time > now()-' + period, chunked=True)
result = client.query(sql)
print(type(result), result['perfkitbenchmarker'].keys())

# https://www.programcreek.com/python/example/107756/influxdb.DataFrameClient
column = next(iter(result))
data   = result[column]
print(column)
# print(data)
# convert utc time to local time
data.index = data.index.tz_convert('Europe/Berlin')
# plotly tries to use utc time first, so remove timezone information:
# https://github.com/plotly/plotly.py/blob/6f9621a611da36f10678c9d9c8c784f55e472429/plotly/utils.py#L263
data.index = data.index.tz_localize(None)
print(type(data))
# return data


Create pandas DataFrame
<class 'collections.defaultdict'> Index(['citation', 'cite_key', 'metric', 'metric_name', 'metric_summary',
       'metric_unit', 'metric_usage', 'official', 'owner', 'product_name',
       'run_number', 'run_uri', 'sample_uri', 'test', 'unit', 'value'],
      dtype='object')
perfkitbenchmarker
<class 'pandas.core.frame.DataFrame'>


In [42]:
df = data
print(df.shape)
df.info()
df['metric'].describe()


(32, 16)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 32 entries, 2020-02-25 04:51:01.912091648 to 2020-02-25 05:02:34.510471168
Data columns (total 16 columns):
citation          8 non-null object
cite_key          8 non-null object
metric            32 non-null object
metric_name       8 non-null object
metric_summary    8 non-null object
metric_unit       8 non-null object
metric_usage      8 non-null object
official          32 non-null object
owner             32 non-null object
product_name      32 non-null object
run_number        8 non-null object
run_uri           32 non-null object
sample_uri        32 non-null object
test              32 non-null object
unit              32 non-null object
value             32 non-null float64
dtypes: float64(1), object(15)
memory usage: 4.2+ KB


count                     32
unique                     3
top       End to End Runtime
freq                      24
Name: metric, dtype: object